In [ ]:
#This code takes as an input a folder of 2D tif masks, it transforms them into a stack, binarizes 
#them, applies a median z-filter to correct skipped frames in the glomeruli segmentation, applies 
#connected components analysis and gives a tif stack and a txt file with glomeruli data as output.

#For large images (Z = 4160) coming from two different cameras (CAM1, CAM2) a different image is saved
#for each camera (otherwise it required too much RAM).

import os

#Folder
imPath = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mm/RL01--X00--Y03--C01'

#File
#resultName = 'Corrected RL01--X00--Y03--C01 mask'
resultName = 'trialMarchingCubes'

visualSlide = 250

#FilterSize (dimensions [Z,Y,X]). The filter analyzes the pixel in its middle
filtSize = [5,1,1]

#Check whether you want to perform connected component analysis or not
conComps = True


#The order in which you want to process both cameras
cams = ['CAM1','CAM2']

#Whether you want only the first cam in "cams" to be processed (because of memory limits)
processOnlyOneCam = True
#------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------




from glob import glob as glob
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import measure
import time

tic0 = time.time()
tic=tic0

imageNamesOrig = glob(imPath+'/*.tif')


      
for CAM in cams:

    #Keep only the names for 'CAM1' or 'CAM2'
    imageNames = [name for name in imageNamesOrig if CAM in name]

    imShape = tiff.imread(imageNames[0]).shape
    numIms = len(imageNames)

    stackDims = np.append(numIms,imShape)

    stack = np.empty (stackDims,dtype = np.uint16)

    slide = -1

    print('\nConverting ',stackDims[0], ' single (',stackDims[1],' x ',stackDims[2], \
          ') yx slides into a zyx stack\n')

    #Save all images from the folder as slides of "stack"
    for imageName in imageNames:
        slide+=1
        im = tiff.imread(imageName)
        #Binarize the image
        im = 1*(im>0)

        #Change bit depth
        im.astype('uint8')
        stack[slide,:,:] = im
        print('Slide ',slide,' of ',stackDims[0],' added')

    #plt.figure()
    #plt.imshow(stack[visualSlide,:,:])
    #plt.title('Binarized stack before z-filtering')    

    #Binarize the stack
    #stack = 1*(stack>0)
    #print('The stack has been binarized')

    #Apply the z-median filter (yz dimensions )
    stack = ndimage.median_filter(stack,size = filtSize)

    print('\nThe z-median filter (shape ',str(filtSize),' has been applied')
    toc = time.time()
    timeStep=toc-tic
    print('Time Z filter: ',timeStep)
    tic = toc
    #plt.figure()
    #plt.imshow(stack[visualSlide,:,:])
    #plt.title('Stack after z-filtering')

    if conComps is True:
        #Extract the connected components from the binary stack

        if (type(stack[0,0,0]) is not 'uint8') and (np.max(stack)<=(2**16-1)):
            stack = stack.astype('uint16')

        print(type(stack[0,0,0]))
        stack = measure.label(stack)

        print('\nThe connected components have been labeled')
        toc = time.time()
        timeStep = toc-tic
        print('Time connected components labelling: ',timeStep)
        tic = toc
        
        conCompData = measure.regionprops(stack) 
        
        print('\nThe connected components have been analyzed')
        toc = time.time()
        timeStep=toc-tic
        print('Time connected components analysis: ',timeStep)
        tic = toc
        
        
    if (type(stack[0,0,0]) is not 'uint8') and (np.max(stack)<=(2**16-1)):
        stack = stack.astype('uint16')

    #Only if there are different cams save the CAM number in the name
    if 'CAM' in imageNamesOrig[0]:
        resultPath = os.path.dirname(imPath) + str(CAM) + '-' + \
        resultName + '-z-filtered(dimsZ'+str(filtSize[0])+'Y'+str(filtSize[1])+'X'+str(filtSize[2])

    else:
        resultPath = os.path.dirname(imPath) +  \
        resultName + '-z-filtered(dimsZ'+str(filtSize[0])+'Y'+str(filtSize[1])+'X'+str(filtSize[2])

    if conComps is True:
        resultPath += '-conComps.tif'
    else:
        resultPath +='.tif'    
        
    tiff.imsave(resultPath, stack, bigtiff=True)
    
    #Free some memory by deleting the stack variable
    
    
    print('\n',str(CAM),' stack has been saved')
    
    #Save the connected components Data:
    #----------------------------------------------------------------
    features = list()
    #First, create the features vector
    for feature in conCompData[0]:
        features.append(feature)

    features.remove('convex_image')
    features.remove('coords')
    features.remove('filled_image')
    features.remove('image')
    features.remove('inertia_tensor')
    features.remove('moments')
    features.remove('moments_central')
    features.remove('moments_normalized')
    features.remove('eccentricity')
    features.remove('moments_hu')
    features.remove('orientation')
    features.remove('perimeter')
    features.remove('perimeter_crofton')
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #Introduce the marching cubes algorithm:
    surfaces = np.zeros(np.max(stack))
    
    #Trying to analyze one surface at a time (for each glomerulus)
    for i in range(1,np.max(stack)+1):
        verts, faces, _, _  = measure.marching_cubes(stack,i)
    
        surfaces[i-1]= measure.mesh_surface_area(verts, faces)        
    print(surfaces)
    print('surfaces.shape: ',surfaces.shape)
    print('\nMarching cubes have been applied')
    toc = time.time()
    timeStep=toc-tic
    print('Time connected components analysis: ',timeStep)
    toc = tic
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    del stack
    
    if 'CAM' in imageNamesOrig[0]:
        conCompDataPath = os.path.dirname(imPath) + str(CAM) + '-' +  \
        'conCompData'+ resultName +'.txt'
    else:
        conCompDataPath = os.path.dirname(imPath) +  \
        'conCompData'+ resultName + '.txt'
    
    with open(conCompDataPath, 'w') as f:
        print('\nWriting connected components data in a .txt file')
        for feature in features:
            f.write(feature)
            f.write(';')
        f.write('\n')
        for component in conCompData:
            for feature in features:
                try:
                    f.write(str(component[feature]))
                except:
                    f.write(' ')
                f.write(';')
            f.write('\n')
    print('\nThe .txt with the connected components data has been saved')
    toc =time.time()
    timeStep = toc-tic
    print('Time writing connected components data in .txt',timeStep,'\n\n')
    
    tic = toc
    #Try to free as much memory as possible
    del feature, features,conCompDataPath
    #-------------------------------------------------------------
    
    #Do not perform the second iteration if there are no cameras in the images or if you only want
    #to process the first camera in "cams"
    if ('CAM' not in imageNamesOrig[0]) or (processOnlyOneCam is True):
        break
        
toc = time.time()        
timeStep = toc-tic0
print('\nTotal execution time: ',timeStep)

In [ ]:
#This code takes each of the tif stacks from a specific folder and saves them as 2D separate images
import os

#Folder with the original images (to get the original names of the slides)
exampleFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/Kidney1_derecho_4.8x_1x3_R&Lshifted1.6mm'

#Folder where the .tif stacks are
inputFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/CAM1'

#Folder where you want to store the individual 2D tifs
outputFolder = 'D:/AAV para enfermedades renales/Stardist glomeruli/Stardist results on whole image/' + \
'Result on large image/'+'/Mask-' + os.path.basename(exampleFolder)



from glob import glob as glob
import tifffile as tiff

#List all the .tif stacks in inputFolder
tifStackNames = glob(inputFolder+'/*.tif')

#For each of them
for name in tifStackNames:
    #Read the current .tif stack
    stack = tiff.imread(name)
    print('Stack ',name,' has been opened')
    #Extract the Side, X and Y coordinates from the name:
    stackCoords = 'RL'+name.split('RL')[1].split(' mask')[0]
    #Get the corresponding camera. cam = 'CAM1' or cam = 'CAM2'
    cam = 'CAM'+name.split('CAM')[1][0]
    
    #Original image names
    origImNames = glob(exampleFolder + '/'+ stackCoords+'/*'+cam+'*')
    
    if not os.path.exists(outputFolder + '/' +stackCoords):
        os.makedirs(outputFolder + '/' +stackCoords)
        print("Directory ",outputFolder,'/',stackCoords,' created')
    #For each of the slides in the .tif stack
    for slide in range(stack.shape[0]):
        fileName = outputFolder+'/'+stackCoords+'/'+os.path.basename(origImNames[slide])
        #Save each slide with the same name as the original one
        tiff.imsave(fileName,stack[slide,:,:])
        print('Slide number ',slide,' saved')
    
    